<a href="https://colab.research.google.com/github/Sopralapanca/Pattern-Recognition/blob/main/Assignment4__Christian_Peluso.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setting the Stage

In [ ]:
%env PYTHONPATH = # /env/python

env: PYTHONPATH=# /env/python


In [ ]:
%%capture
!wget https://repo.anaconda.com/miniconda/Miniconda3-py38_4.12.0-Linux-x86_64.sh
!chmod +x Miniconda3-py38_4.12.0-Linux-x86_64.sh
!./Miniconda3-py38_4.12.0-Linux-x86_64.sh -b -f -p /usr/local
!conda update -y conda

In [ ]:
import sys
sys.path.append('/usr/local/lib/python3.8/site-packages')

In [ ]:
%%capture
!conda create -y -n myenv python=3.6

In [ ]:
!nvidia-smi --query-gpu=driver_version --format=csv

driver_version
525.85.12


In [ ]:
%%shell
eval "$(conda shell.bash hook)"
conda activate myenv
conda install -y cudatoolkit=10.0
conda install -y cudnn=7.3.1
pip3 install -q tensorflow-gpu==1.15

Solving environment: \ | done


==> WARNING: A newer version of conda exists. <==
  current version: 4.12.0
  latest version: 23.3.1

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /usr/local/envs/myenv

  added / updated specs:
    - cudatoolkit=10.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    cudatoolkit-10.0.130       |                0       261.2 MB
    ------------------------------------------------------------
                                           Total:       261.2 MB

The following NEW packages will be INSTALLED:

  cudatoolkit        pkgs/main/linux-64::cudatoolkit-10.0.130-0



cudatoolkit-10.0.130 | 261.2 MB  | : 100% 1.0/1 [00:06<00:00,  6.46s/it]               
Preparing transaction: - done
Verifying transaction: | done
Executing transaction: - done
Solving environment: | fa

In [ ]:
multiline_str = ("import tensorflow as tf\n"
"print(tf. __version__)\n"
"print(tf.__path__ )\n"
"if tf.test.gpu_device_name():\n"
"  print('Default GPU Device:{}'.format(tf.test.gpu_device_name()))\n"
"else:\n"
"  print('Please install GPU version of TF')")

In [ ]:
with open('test.py', 'w') as f:
    f.write(multiline_str)

In [ ]:
%%shell
eval "$(conda shell.bash hook)"
conda activate myenv
python /content/test.py

1.15.0
['/usr/local/envs/myenv/lib/python3.6/site-packages/tensorflow_core/python/keras/api/_v1', '/usr/local/envs/myenv/lib/python3.6/site-packages/tensorflow_estimator/python/estimator/api/_v1', '/usr/local/envs/myenv/lib/python3.6/site-packages/tensorflow_core', '/usr/local/envs/myenv/lib/python3.6/site-packages/tensorflow_core/_api/v1']
2023-05-12 17:15:45.677143: I tensorflow/core/platform/cpu_feature_guard.cc:142] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
2023-05-12 17:15:45.683360: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2299995000 Hz
2023-05-12 17:15:45.683557: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x558cc36b5500 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2023-05-12 17:15:45.683586: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version
2023-05-12 17:15:45.687054: I tensorflow/strea

The stage is set, now we can start to import the repository and the dataset...

## Data extraction

In [ ]:
!git clone https://github.com/sherjilozair/char-rnn-tensorflow.git

Cloning into 'char-rnn-tensorflow'...
remote: Enumerating objects: 404, done.
remote: Total 404 (delta 0), reused 0 (delta 0), pack-reused 404
Receiving objects: 100% (404/404), 508.45 KiB | 4.20 MiB/s, done.
Resolving deltas: 100% (238/238), done.


In [ ]:
!gdown "1AvpkoouQhcmoYzqcbkQd-XAawOC_0-G6"
!gdown "1RZnghfnuVRtUxybFiyglLDnWZp4srg6M"

Downloading...
From: https://drive.google.com/uc?id=1AvpkoouQhcmoYzqcbkQd-XAawOC_0-G6
To: /content/lercio_headlines.csv
100% 487k/487k [00:00<00:00, 108MB/s]
Downloading...
From: https://drive.google.com/uc?id=1RZnghfnuVRtUxybFiyglLDnWZp4srg6M
To: /content/it.parquet
100% 4.87M/4.87M [00:00<00:00, 125MB/s]


In [ ]:
import pandas as pd
import numpy as np
import os

PATH_TO_DATA="/content/char-rnn-tensorflow/data/"

lercio = pd.read_csv("/content/lercio_headlines.csv", header=None)
corpus = pd.read_parquet("/content/it.parquet")
corpus['x']=corpus['x'].str.replace(r"[-!$%^&*()_+|~=`{}\[\]:;'<>?\/]",'', regex=True)

try :
  os.mkdir(PATH_TO_DATA+"lercioheadlines/")
except:
  pass
try :
  os.mkdir(PATH_TO_DATA+"corpus/")
except:
    pass

np.savetxt(PATH_TO_DATA+'lercioheadlines/input.txt', lercio.values, fmt='%s')
np.savetxt(PATH_TO_DATA+'corpus/input.txt', corpus.values, fmt='%s', 
           delimiter=' ', newline='\n', header='')

In [ ]:
os.chdir('char-rnn-tensorflow')

## Random Search

Here we are searching through the space of hyperparameters, reducing the time spent in the model selection. For simplicity here are listen only two models, but originally the RS were performed in a stream of 10 models, then the checkpoint of the best models were loaded and compared in generator skills

In [ ]:
parameters = {'model':['gru', 'lstm','nas'],
              'rnn_size':[96,128,256],
              'seq_length':range(130,140,2), # The average number of chars in a title is 72 so we span the memory of the RNN at max this number
              'batch_size':range(200,250,10),
              'num_layers':range(2,4,1),
              'output_keep_prob':np.arange(0.7,1,0.1),
              'input_keep_prob':np.arange(0.8,1,0.1),
              'grad_clip':np.arange(3,8,1),
              'learning_rate':np.arange(0.007,0.02,0.003),
              'decay_rate':np.arange(0.9,1,0.02)}

In [ ]:
import random
random_search = ''
pars = {}
for run in range(0,2):
  try :
    os.mkdir(f"./save/save{run}")
  except:
    pass
  for i, k in enumerate(parameters.keys()):
    r = random.randint(0,len(parameters[k]))
    pars[i]=(parameters[k][r-1])
  random_search = random_search + f'python train.py --data_dir=./data/corpus/ --save_dir=./save/save{run} --model={pars[0]} --num_epochs=50 --rnn_size={pars[1]} --seq_length={pars[2]} --batch_size={pars[3]} --num_layers={pars[4]} --output_keep_prob={pars[5]} --input_keep_prob={pars[6]} --grad_clip={pars[7]} --learning_rate={pars[8]} --decay_rate={pars[9]}\n'
print(random_search)

python train.py --data_dir=./data/corpus/ --save_dir=/content/drive/Shareddrives/Materials\ UNI/UNIPI/ISPR/Midterm3\ Assignment_4/save/save0 --model=nas --num_epochs=50 --rnn_size=256 --seq_length=130 --batch_size=200 --num_layers=2 --output_keep_prob=0.9999999999999999 --input_keep_prob=0.8 --grad_clip=5 --learning_rate=0.019 --decay_rate=0.92
python train.py --data_dir=./data/corpus/ --save_dir=/content/drive/Shareddrives/Materials\ UNI/UNIPI/ISPR/Midterm3\ Assignment_4/save/save1 --model=nas --num_epochs=50 --rnn_size=256 --seq_length=136 --batch_size=210 --num_layers=2 --output_keep_prob=0.8999999999999999 --input_keep_prob=0.8 --grad_clip=6 --learning_rate=0.007 --decay_rate=0.9800000000000001



In [ ]:
with open('file', 'w') as f:
    f.write(random_search)

In [ ]:
%%shell
eval "$(conda shell.bash hook)"
conda activate myenv
bash file

Output streaming troncato alle ultime 5000 righe.
19135/20850 (epoch 45), train_loss = 1.925, time/batch = 0.745
19136/20850 (epoch 45), train_loss = 1.928, time/batch = 0.689
19137/20850 (epoch 45), train_loss = 1.954, time/batch = 0.684
19138/20850 (epoch 45), train_loss = 1.900, time/batch = 0.728
19139/20850 (epoch 45), train_loss = 1.907, time/batch = 0.683
19140/20850 (epoch 45), train_loss = 1.920, time/batch = 0.719
19141/20850 (epoch 45), train_loss = 1.907, time/batch = 0.729
19142/20850 (epoch 45), train_loss = 1.906, time/batch = 0.725
19143/20850 (epoch 45), train_loss = 1.912, time/batch = 0.710
19144/20850 (epoch 45), train_loss = 1.940, time/batch = 0.680
19145/20850 (epoch 45), train_loss = 1.929, time/batch = 0.718
19146/20850 (epoch 45), train_loss = 1.920, time/batch = 0.708
19147/20850 (epoch 45), train_loss = 1.920, time/batch = 0.682
19148/20850 (epoch 45), train_loss = 1.951, time/batch = 0.670
19149/20850 (epoch 45), train_loss = 1.927, time/batch = 0.651
19150

## Fine tuning 
Fine tune the best model on the Lercio Headlines Dataset, in such a way that we can perform transfer learning keeping the sintax of Italian.

In [ ]:
!gdown "1D0b_4pS2TdYcTMpruTv1fH1sejWs5kYX"

Downloading...
From: https://drive.google.com/uc?id=1D0b_4pS2TdYcTMpruTv1fH1sejWs5kYX
To: /content/char-rnn-tensorflow/FinalModel.zip
100% 175M/175M [00:02<00:00, 71.5MB/s]


In [ ]:
%%capture

!unzip FinalModel.zip

In [ ]:
import pickle

with open("FinalModel/config.pkl","rb") as file_handle:
    retrieved_data = pickle.load(file_handle)
    print(retrieved_data)
retrieved_data.save_dir = './FinalModel'
print(retrieved_data)
with open("FinalModel/config.pkl","wb") as file:
  pickle.dump(retrieved_data, file)

Namespace(data_dir='./data/lercioheadlines/', save_dir='/content/drive/Shareddrives/Materials UNI/UNIPI/ISPR/Midterm3 Assignment_4/save/save0', log_dir='logs', save_every=1000, init_from=None, model='nas', rnn_size=256, num_layers=2, seq_length=130, batch_size=200, num_epochs=50, grad_clip=5.0, learning_rate=0.019, decay_rate=0.92, output_keep_prob=0.9999999999999999, input_keep_prob=0.8, vocab_size=145)
Namespace(data_dir='./data/lercioheadlines/', save_dir='./FinalModel', log_dir='logs', save_every=1000, init_from=None, model='nas', rnn_size=256, num_layers=2, seq_length=130, batch_size=200, num_epochs=50, grad_clip=5.0, learning_rate=0.019, decay_rate=0.92, output_keep_prob=0.9999999999999999, input_keep_prob=0.8, vocab_size=145)


In [ ]:
with open("FinalModel/checkpoint") as file:
  retrieved = file.read()
print(retrieved)
retrieved = retrieved.replace('model_checkpoint_path: "/content/drive/Shareddrives/Materials UNI/UNIPI/ISPR/Midterm3 Assignment_4/save/save0/model.ckpt-849', 'model_checkpoint_path: "/FinalModel/model.ckpt-849"')
retrieved = retrieved.replace("/content/drive/Shareddrives/Materials UNI/UNIPI/ISPR/Midterm3 Assignment_4/save/save0/model.ckpt-0", "FinalModel/model.ckpt-0")
retrieved = retrieved.replace('all_model_checkpoint_paths: "/content/drive/Shareddrives/Materials UNI/UNIPI/ISPR/Midterm3 Assignment_4/save/save0/model.ckpt-849', 'all_model_checkpoint_paths: "/FinalModel/model.ckpt-849"')
print(retrieved)
with open("FinalModel/checkpoint", "w") as file:
  retrieved = file.write(retrieved)

model_checkpoint_path: "/content/drive/Shareddrives/Materials UNI/UNIPI/ISPR/Midterm3 Assignment_4/save/save0/model.ckpt-849"
all_model_checkpoint_paths: "/content/drive/Shareddrives/Materials UNI/UNIPI/ISPR/Midterm3 Assignment_4/save/save0/model.ckpt-0"
all_model_checkpoint_paths: "/content/drive/Shareddrives/Materials UNI/UNIPI/ISPR/Midterm3 Assignment_4/save/save0/model.ckpt-849"

model_checkpoint_path: "/FinalModel/model.ckpt-849""
all_model_checkpoint_paths: "FinalModel/model.ckpt-0"
all_model_checkpoint_paths: "/FinalModel/model.ckpt-849""



In [ ]:
%%shell
eval "$(conda shell.bash hook)"
conda activate myenv
python train.py --data_dir=./data/lercioheadlines/ --save_dir=/FinalModel/ --model=nas --num_epochs=50 --rnn_size=256 --seq_length=130 --batch_size=200 --num_layers=2 --output_keep_prob=0.9999999999999999 --input_keep_prob=0.8 --grad_clip=5 --learning_rate=0.019 --decay_rate=0.92

reading text file
Instructions for updating:
This class is equivalent as tf.keras.layers.StackedRNNCells, and will be replaced by that in Tensorflow 2.0.



Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Please use `layer.add_weight` method instead.

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where




2023-05-12 12:58:38.959019: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2023-05-12 12:58:38.982204: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:983] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-05-12 12:58:38.982457: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1618] Found device 0 with properties: 
name: Tesla T4 major: 7 minor: 5 memoryClockRate(GHz): 1.59
pciBusID: 0000:00

# Generation of Headlines

Here we are modifying the `sample.py` in order to avoid to print all the TensorFlow warnings.

In [ ]:
with open('sample.py', 'r') as f:
    script = f.readlines()
script.insert(10, "import tensorflow as tf\n"
                  "tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)\n" )
with open('sample2.py', 'a') as f:
  for line in script:
    f.write(line)

### sample = 0
The flag `sample=0` permits to run the prediction on max value each step of the prediction, this means that the model will pick at each time-step $t$ the value 
$$\max_{v\in V} P(V_t=v|V_{t-1}=v_{t-1})$$

In [ ]:
%%shell
eval "$(conda shell.bash hook)"
conda activate myenv

export TF_CPP_MIN_LOG_LEVEL="3" && python sample2.py --sample=0 --save_dir=/FinalModel/

 più contro i propri di cantiere
Scoperto il primo casa di cartellino di cantiere di casa di cantiere
Scoperto il primo casa di casa di casa di casa di casa di cantiere
Scoperto il primo casa di casa di casa di casa di casa di casa di casa di casa di casa di casa di casa di casa di casa di casa di casa di casa di casa di casa di casa di casa di casa di casa di casa di casa di casa di casa di casa di casa di casa di casa di casa di casa di casa di casa di casa di casa di casa di casa di casa di ca


As we can see the prediction keep enrolling a loop of characters, so the sampling process is not satisfying.

### sample = 1
The flag `sample=1` permits to run the prediction on the weighted average on the precedent value so distribution will be described by a Markov Chain conditioned only by the n-precedent characters, where n is the seq_length of the RNN memory: 
$$\prod_{i=0}^{seq\_length} P(v_{t-i}|v_{t-i-1})$$

In [ ]:
%%shell
eval "$(conda shell.bash hook)"
conda activate myenv

export TF_CPP_MIN_LOG_LEVEL="3" && python sample2.py --sample=1 --save_dir=/FinalModel/

 sfigliata di Hilano in Wenneri
Clancio e latto in adiuto con la cavallo: “Se”, ento di cone di guardato e Berlusconi: “Basta 500mila buss muore con un tusto più di Alfan
Blello diventa “Arcere compie contro i prossima passa”
SESMIO, IL RABOR A MANTO – ANDERETTE, LA PARDONA E PRUDO FAPIOCHITE-IL MORA
Prepog sulla Fi la buca del moro e lancia2
Merce in rotto: la vorigistra sbagliato corsa: “Con i Cabrio Francesco Basinis funi in auto d’auto e ingeglia al calcer.
Padre viale, trattici inseg salta l


As we can see with the weighted average there are no more loops, but the probability of encountering some sequence of characters is lower therefore the model will have more difficulties in predict the next one based on the lower neuron activations.

### sample = 2

The flag `sample=2` permits to run the prediction on the weighted average of the precedent value when it is a space, in such a way that new space will be sampled on the max values of the first characters per each new token, so distribution will be described by a Markov Chain conditioned only by the n-precedent characters and on the max value when the precedent char is a space so the model will predict with: \begin{cases}
    \max_{v\in V} P(V_t=v|V_{t-1}=v_{t-1}),& \text{if } v_{t-1} = 0\\
    \prod_{i=0}^{seq\_length} P(v_t-i|v_{t-i-1}),              & \text{otherwise}
\end{cases}

In [ ]:
%%shell
eval "$(conda shell.bash hook)"
conda activate myenv

export TF_CPP_MIN_LOG_LEVEL="3" && python sample2.py --sample=2 --save_dir=/FinalModel/

 si scopre sulla manica di quello con un testa a segreto di partita di solo la prima non la manico di partita di cantiere
Scoperto anche a casa sulla carta di Papa per aver discorso Marino
Scoperto ricorda sulla scopre Maria si proprio esperienza per la vita di Maria Jong per la prima bambino per diventa a #stato di Papa e ’0 con la prima discorsi ma diventa la manico di partita
Scoperto diventa un vince il primo vince il nuovo legge in 7 negati finisce a martito di Renzi vince a Trump
Marino di 


This surely is the best way to sample since the model will create words that are highly common through the corpus so it's mostly confident in the prediction of new characters, but there is a bit of loops in the creation of the first sequences.

Now we try to use the flag `prime_text` that will aim to start the model to predict the next sequence of characters basing on the entered string.


In [ ]:
%%shell
eval "$(conda shell.bash hook)"
conda activate myenv

export TF_CPP_MIN_LOG_LEVEL="3" && python sample2.py --save_dir=/FinalModel/

Anno 2050 primate si evolvere il primo vince a ritrova per il nuovo Facebook si accordo di distrare di Allarme e Berlusconi tra avere ricorda un casa e quando ma zio contro il regalino di essere alla scopre con allarme a fare sulla fare alla discorsi per la prima testa per ricorda e inventa la vita la nonna di Jolie 5 contro i senza per aver bambini in jusa per ore sulla fare ma non avere essere alla discorsi è un giorno di Casaleggio “Salvini a un altro giorni di vende in discontare un giorno a scopre la manico a casa di


As we can observe the model kept some sort of grammatic in the first part of the generated sequence and even a little of semantic if we want, since the primate could use for the first time Facebook to win some sort of competition.

As we have stated the sintax can be some sort of understood by the model, after heavy usage of training the memory of the cells on "big" corpus and then the fine-tuning permits to shift the probability of the characters prediction to the Lercio kind of style...
We think there is plenty of better models to deal with this kind of task, but even a little more complex one like the Hierarchical-RNN could smash this network having a larger receptive field that could permit to keep the semantic information more easily.